# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [53]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [54]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,18.99,58,100,5.14,US,1675546926
1,1,georgetown,5.4112,100.3354,25.96,88,20,3.09,MY,1675546926
2,2,roald,62.5833,6.1333,6.08,70,75,7.72,NO,1675546545
3,3,inhambane,-23.8650,35.3833,24.36,86,9,3.79,MZ,1675546920
4,4,rikitea,-23.1203,-134.9692,26.08,74,2,6.17,PF,1675546547


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [56]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,18.99,58,100,5.14,US,1675546926
1,1,georgetown,5.4112,100.3354,25.96,88,20,3.09,MY,1675546926
2,2,roald,62.5833,6.1333,6.08,70,75,7.72,NO,1675546545
3,3,inhambane,-23.8650,35.3833,24.36,86,9,3.79,MZ,1675546920
4,4,rikitea,-23.1203,-134.9692,26.08,74,2,6.17,PF,1675546547


In [60]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21; Wind speed less than 4.5 m/s; Zero cloudiness
city_data_df_narrow = city_data_df[city_data_df['Max Temp'] <27]
city_data_df_narrow = city_data_df_narrow[city_data_df_narrow['Max Temp'] >21]
city_data_df_narrow = city_data_df_narrow[city_data_df_narrow['Wind Speed']<4.5]
city_data_df_narrow = city_data_df_narrow[city_data_df_narrow['Cloudiness']==0]

# Drop any rows with null values
city_data_df_narrow = city_data_df_narrow.dropna()

# Display sample data
city_data_df_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,durban,-29.8579,31.0292,24.45,88,0,0.89,ZA,1675546863
93,93,port macquarie,-31.4333,152.9167,22.16,62,0,1.03,AU,1675546975
304,304,pisco,-13.7000,-76.2167,24.03,83,0,4.12,PE,1675547078
378,378,cabo san lucas,22.8909,-109.9124,26.61,47,0,3.60,MX,1675547142
523,523,isla vista,34.4133,-119.8610,22.22,63,0,2.06,US,1675547461


### Step 3: Create a new DataFrame called `hotel_df`.

In [61]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrow[['City', 'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']= ' '
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,durban,ZA,-29.8579,31.0292,88,
93,port macquarie,AU,-31.4333,152.9167,62,
304,pisco,PE,-13.7000,-76.2167,83,
378,cabo san lucas,MX,22.8909,-109.9124,47,
523,isla vista,US,34.4133,-119.8610,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation"
limit = 2

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}
    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
durban - nearest hotel: Tudor
port macquarie - nearest hotel: Town Beach Motor Inn
pisco - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
isla vista - nearest hotel: The Club & Guest House


,City,Country,Lat,Lng,Humidity,Hotel Name
53,durban,ZA,-29.8579,31.0292,88,Tudor
93,port macquarie,AU,-31.4333,152.9167,62,Town Beach Motor Inn
304,pisco,PE,-13.7000,-76.2167,83,No hotel found
378,cabo san lucas,MX,22.8909,-109.9124,47,Comfort Rooms
523,isla vista,US,34.4133,-119.8610,63,The Club & Guest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols='all'
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)